#### Akkadian → English Translation Baseline (血肉化版)
**実行環境:** ローカル (.venv, Python 3.10.13)  
**提出:** Kaggle Notebook または CSVアップロード  
**注意:** Kaggleでは `/kaggle/input/...` のパスを使用してください（ローカルでは `data/deep-past/`）

---

In [1]:
# import os
# print(os.listdir("/kaggle/input/datasets/oreo01mofu/byt5-base-kaggle/byt5-base"))


### なぜ AMP？

### 速度 約1.7倍

### VRAM削減

### byt5-baseなら必須レベル

In [2]:
# ============
# 1. Imports
# ============
# 基本ライブラリ
import os
import re
import pandas as pd
from sklearn.model_selection import KFold  # Cross Validation

# PyTorch
import torch
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler  # AMP用

# HuggingFace Transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 進捗表示
from tqdm.auto import tqdm

## 2. 設定（Config）

ローカル実行用に相対パス指定

Kaggle用には ../input/deep-past-initiative-machine-translation/ に変更可能

In [3]:
# =========================
# 2. Config
# =========================

MODEL_PATH = "/kaggle/input/datasets/oreo01mofu/byt5-base-kaggle/byt5-base"

DATA_DIR = "/kaggle/input/deep-past-initiative-machine-translation"
TRAIN_PATH = os.path.join(DATA_DIR, "train.csv")
TEST_PATH  = os.path.join(DATA_DIR, "test.csv")

# ===== Training Config =====
BATCH_SIZE = 2
MAX_LENGTH = 256
LR = 1e-5
EPOCHS = 1
# N_SPLITS = 5 # CV分割数

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# T5系は prefix 必須
PREFIX = "translate Akkadian to English: "

print("Device:", DEVICE)


Device: cuda


### 3. データ確認（EDA）

まずはデータを理解してからモデルに流す

In [4]:
train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)

print(train_df.shape, test_df.shape)
print(train_df.columns)
train_df.head()

# 文の長さや欠損を確認
train_df['transliteration_len'] = train_df['transliteration'].astype(str).apply(len)
train_df['english_len'] = train_df['translation'].astype(str).apply(len)
train_df[['transliteration_len','english_len']].describe()


(1561, 3) (4, 5)
Index(['oare_id', 'transliteration', 'translation'], dtype='object')


,transliteration_len,english_len
count,1561.000000,1561.000000
mean,426.459321,499.723895
std,269.688193,466.365112
min,21.000000,6.000000
25%,204.000000,172.000000
50%,365.000000,383.000000
75%,632.000000,681.000000
max,932.000000,3895.000000


In [5]:
# ============
# 4. Tokenizer & Model
# ============
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH,
                                       local_files_only=True)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH,
                                             local_files_only=True).to(DEVICE)
model.eval()

# scaler = GradScaler()



2026-02-13 09:23:58.117911: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770974638.302901      23 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770974638.351884      23 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770974638.781190      23 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770974638.781225      23 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770974638.781228      23 computation_placer.cc:177] computation placer alr

T5ForConditionalGeneration(
  (shared): Embedding(384, 1536)
  (encoder): T5Stack(
    (embed_tokens): Embedding(384, 1536)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1536, out_features=768, bias=False)
              (k): Linear(in_features=1536, out_features=768, bias=False)
              (v): Linear(in_features=1536, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=1536, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1536, out_features=3968, bias=False)
              (wi_1): Linear(in_features=1536, out_features=3968, bias=False)
              (

### 5. Dataset定義

テストデータをモデルが食べられる形式に変換

DataLoaderでバッチ処理
### なぜ -100 が必要？

### T5は padding まで loss 計算する仕様。これをやらないとBLEUが伸びない。

In [6]:
# ============
# 5. Dataset 定義
# ============
# 学習用
class TrainDataset(Dataset):
    def __init__(self, df, tokenizer):
        # 入力にPREFIXを付加（T5系はタスク指示が必要）
        self.inputs = [PREFIX + str(t) for t in df["transliteration"].astype(str)]
        self.targets = df["translation"].astype(str).tolist()
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        # ========================
        # 入力文のトークナイズ
        # ========================
        input_encoded = self.tokenizer(
            self.inputs[idx],
            max_length=MAX_LENGTH,
            padding="max_length",     # 長さを固定（バッチ処理のため）
            truncation=True,          # 長すぎる場合は切る
            return_tensors="pt"
        )
        
        # ========================
        # 正解文のトークナイズ
        # ========================
        target_encoded = self.tokenizer(
            self.targets[idx],
            max_length=MAX_LENGTH,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        # ========================
        # ★重要修正ポイント★
        # T5系では pad_token_id を -100 にしないと
        # loss計算にパディングが含まれてしまう
        # ========================
        labels = target_encoded["input_ids"].squeeze(0)
        labels[labels == tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_encoded["input_ids"].squeeze(0),
            "attention_mask": input_encoded["attention_mask"].squeeze(0),
            "labels": labels
        }


# 推論用（変更なしでOK）
class InferenceDataset(Dataset):
    def __init__(self, df, tokenizer):
        # 推論では正解文が存在しない
        self.texts = [PREFIX + str(t) for t in df["transliteration"].astype(str).tolist()]
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        encoded = self.tokenizer(
            self.texts[idx],
            max_length=MAX_LENGTH,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoded["input_ids"].squeeze(0),
            "attention_mask": encoded["attention_mask"].squeeze(0)
        }


## 学習
### なぜCVが必要？

BLEUが不安定なため、単発学習は危険。
### データローダーはfoldごと

In [7]:
# =========================
# 7. Simple Training (安定確認用)
# =========================

# 🔥 CVは一旦やめる（安定確認フェーズ）
train_dataset = TrainDataset(train_df, tokenizer)
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

# モデルを再初期化（重要）
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH).to(DEVICE)

# 勾配爆発防止のため低めLR
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

for epoch in range(EPOCHS):

    print(f"\n===== Epoch {epoch+1} =====")
    model.train()

    total_loss = 0

    for batch in tqdm(train_loader):

        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)

        optimizer.zero_grad()

        # 🔥 AMPは使わない（まず安定させる）
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss

        # NaNチェック（超重要）
        if torch.isnan(loss):
            print("⚠️ NaN detected. Stopping training.")
            break

        loss.backward()

        # 勾配爆発防止
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Average Train Loss: {avg_loss:.4f}")



===== Epoch 1 =====


  0%|          | 0/781 [00:00<?, ?it/s]

Average Train Loss: 1.6523


In [8]:
# =========================
# Inference DataLoader 再定義
# =========================

test_dataset = InferenceDataset(test_df, tokenizer)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print("Test loader ready.")


Test loader ready.


In [9]:
# ============
# 8. 推論ブロック
# ============
all_predictions = []
torch.set_grad_enabled(False)
model.eval()

with torch.inference_mode():
    for batch in tqdm(test_loader):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)

        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=MAX_LENGTH,
            num_beams=5,          # Beam強化
            length_penalty=1.0,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        all_predictions.extend([d.strip() for d in decoded])

  0%|          | 0/2 [00:00<?, ?it/s]

### 9. Postprocess

BLEUなど評価に影響するノイズを軽く除去

In [10]:
def postprocess(text: str) -> str:
    if not isinstance(text, str) or len(text.strip()) == 0:
        return "broken text"
    return re.sub(r"\s+", " ", text).strip()

predictions = [postprocess(p) for p in all_predictions]


In [11]:
# ============
# 8. Submission
# ============

#BLEU において「意味が正しいのにスコアが下がる」例
#

# 提出用データフレームを作成する
submission = pd.DataFrame({
    "id": test_df["id"],
    "translation": predictions
})

# 提出用データフレームを保存する
submission.to_csv("submission.csv", index=False)

# 提出用データフレームを保存したことを表示する
print("✅ submission.csv saved")


✅ submission.csv saved


## 可視化

In [12]:
model.eval()

sample_text = train_df.iloc[0]["transliteration"]
input_text = PREFIX + sample_text

inputs = tokenizer(
    input_text,
    return_tensors="pt",
    truncation=True,
    max_length=MAX_LENGTH
).to(DEVICE)

with torch.no_grad():
    outputs = model.generate(**inputs, max_length=MAX_LENGTH)

pred = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("INPUT:", sample_text[:200])
print("GT:", train_df.iloc[0]["translation"][:200])
print("PRED:", pred[:200])


INPUT: KIŠIB ma-nu-ba-lúm-a-šur DUMU ṣí-lá-(d)IM KIŠIB šu-(d)EN.LÍL DUMU ma-nu-ki-a-šur KIŠIB MAN-a-šur DUMU a-ta-a 0.33333 ma-na 2 GÍN KÙ.BABBAR SIG₅ i-ṣé-er PUZUR₄-a-šur DUMU a-ta-a a-lá-ḫu-um i-šu iš-tù ḫ
GT: Seal of Mannum-balum-Aššur son of Ṣilli-Adad, seal of Šu-Illil son of Mannum-kī-Aššur, seal of Puzur-Aššur son of Ataya. Puzur-Aššur son of Ataya owes 22 shekels of good silver to Ali-ahum. Reckoned f
PRED: received by Manubalum-Aššur and Manubalum-Aššur and Manubalum-Aššur and Manubalum-Aššur and Manubalum-Aššur and Manubalum-Aššur and Manubalum-Aššur and Manubalum-Aššur and 


##　復習用
### 🔥 Kaggle Seq2Seq 実装理解チェック

## 1. Datasetは何を返している？
Q: TrainDatasetは何を返す？

A: input_ids / attention_mask / labels の3つ。モデルにそのまま渡せる辞書形式。

---

## 2. なぜ labels の padding を -100 にする？
Q: なぜ pad_token_id を -100 にする？

A: T5は -100 を無視する仕様。paddingまでloss計算されるのを防ぐため。

---

## 3. attention_mask の役割は？
Q: attention_mask は何をしている？

A: padding部分を無視させるためのマスク。

---

## 4. model(..., labels=labels) で何が起きる？
Q: labels を渡すと何が返る？

A: 自動で CrossEntropyLoss を計算して loss を返す。

---

## 5. AMPとは？
Q: autocast と GradScaler は何のため？

A: 半精度(fp16)で計算して高速化＋メモリ削減するため。

---

## 6. Cross Validationの目的は？
Q: なぜCVを使う？

A: データ分割の偶然性を減らして汎化性能を安定させるため。

---

## 7. model.generate() は何をしている？
Q: generate は何をしている？

A: デコーダでトークンを1つずつ予測しながら文章を生成している。推論のみ！

---

## 8. num_beams とは？
Q: beam search は何をしている？

A: 複数候補を並列に探索してより良い翻訳を選ぶ。

---

## 9. OOMはなぜ起きる？
Q: なぜメモリが足りなくなる？

A: バッチサイズ × シーケンス長 × モデルサイズ が大きすぎるから。

---

## 10. 何を減らせばメモリが減る？
A:
- BATCH_SIZEを減らす
- MAX_LENGTHを減らす
- モデルサイズを小さくする
- gradient_checkpointingを使う
